In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

import shutil
directory_out = os.path.join(path_root, './file/3.part')
if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)
import re

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = =  = = =  = = =

import itertools

# = = =  = = =  = = =

import html as html_lib

# = = =  = = =  = = =

directory_in = os.path.join(path_root, './file/2.url')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
for file in list_file:

    # = = =  = = =  = = =

    print('Data Loading...')
    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                        header=0,
                                                                        dtype=str).fillna('')
    print('Loading Done !')
    print()

    df_input['No'] = df_input['No'].astype(int)

    crawler_remain = len(df_input)

    print(f'总数量：{crawler_remain}')
    print()

    # = = =  = = =  = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, crawler_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = =  = = =  = = =

        try:
            crawler_retry = 0
            while True:
                crawler_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        dict_ = json.loads(html.xpath('//script[@type="application/json"]/text()')[0])
                        if dict_:
                            break
                except requests.exceptions.ProxyError as e:
                    if '456 CN Client Forbidden' in str(e):
                        print('Long live the People\'s Republic of China!')
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = =  = = =  = = =

            dict_src, list_img = {}, dict_['images']
            for i, img in enumerate(list_img):
                dict_src[str(i)] = f'''https:{img['main']['url'].strip()}''' if img['main']['url'].strip().startswith('//') else img['main']['url'].strip()

            # = = =  = = =  = = =

            list_dict_vehicle, list_dict_fitment = [], dict_['fitment']
            for dict_fitment in list_dict_fitment:
                dict_fitment.pop('idx')
                list_key = list(dict_fitment.keys())
                list_value = [dict_fitment[key] if type(dict_fitment[key]) == list else [dict_fitment[key]] for key in list_key]

                product_combination = itertools.product(*list_value)
                for combination in product_combination:
                    list_dict_vehicle.append(dict(zip(list_key, combination)))

            # = = =  = = =  = = =

            subtitle = html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()')[0].strip() if html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()') else ''

            # = = =  = = =  = = =

            dict_key, list_li = {}, html.xpath('//h2[@class="product-details-title"]/following-sibling::ul[@class="field-list"][1]/li')
            for li in list_li:
                key = li.xpath('./span[@class="list-label"]')[0]
                key = etree.tostring(key).decode('utf-8')
                key = re.sub(r'<.*?>', '', key)
                key = re.sub(r' +', ' ', key)
                key = html_lib.unescape(key)
                key = key.strip().removesuffix(':').strip()
                if key == 'Brand':
                    continue
                elif key in dict_key:
                    dict_key[key] += 1
                else:
                    dict_key[key] = 1

            dict_detail, dict_count = {}, {}
            for i, li in enumerate(list_li):
                key = li.xpath('./span[@class="list-label"]')[0]
                key = etree.tostring(key).decode('utf-8')
                key = re.sub(r'<.*?>', '', key)
                key = re.sub(r' +', ' ', key)
                key = html_lib.unescape(key)
                key = key.strip().removesuffix(':').strip()
                if key == 'Brand':
                    list_src = li.xpath('./span[@class="list-value"]/div[@class="part-manufacturer"]/div[@class="manufacturer-logos"]/img/@src')
                    list_src = ['.'.join(src.split('/')[-1].split('.')[:-1]).title().strip() for src in list_src]
                    dict_detail[str(i)] = {key: ';'.join(list_src)}
                else:
                    value = li.xpath('./span[contains(@class, "list-value")]')[0]
                    value = etree.tostring(value).decode('utf-8')
                    value = re.sub(r'<.*?>', '', value)
                    value = re.sub(r' +', ' ', value)
                    value = html_lib.unescape(value)
                    if dict_key[key] == 1:
                        dict_detail[str(i)] = {key: value.strip()}
                    else:
                        if key in dict_count:
                            dict_count[key] += 1
                        else:
                            dict_count[key] = 1
                        dict_detail[str(i)] = {f'{key}-{dict_count[key]}': value.strip()}

            # = = =  = = =  = = =

            dict_correct_temp = {'No': crawler_series['No'],
                                'Catalog': dict_['catalog'].title().strip(),
                                'Part Number': dict_['sku'].strip(),
                                'Title': dict_['title'].strip(),
                                'Subtitle': subtitle,
                                'Price': dict_['price'],
                                'Picture': '',
                                'Url': crawler_series['Url'],
                                'Json_Src': json.dumps(dict_src),
                                'Json_Detail': json.dumps(dict_detail),
                                'Json_CPN': f'''{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'''}

            # = = =  = = =  = = =

            list_dict_correct_temp = [{**dict_correct_temp, **dict_vehicle} for dict_vehicle in list_dict_vehicle]

            # = = =  = = =  = = =

            with semaphore_correct:
                list_dict_correct.extend(list_dict_correct_temp)

            # = = =  = = =  = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = =  = = =  = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and crawler_remain == 1 or crawler_remain % 30 == 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{file.removesuffix('.xlsx')} ({(list_file.index(file) + 1) / len(list_file) * 100:.2f}%)] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            crawler_remain -= 1

    # = = =  = = =  = = =

    pool.map(crawler, df_input.iterrows())

    print('Data Outputting...')
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct)
        df_correct['year'] = df_correct['year'].astype(int)
        df_correct = df_correct.sort_values(by=['No', 'Part Number', 'make', 'model', 'year', 'trims', 'engines'],
                                            ascending=[True, True, True, True, False, True, True],
                                            ignore_index=True)
        df_correct.to_excel(os.path.join(directory_out, f'''./{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
    print('Output Finished !')
    print()
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                            ascending=[True],
                                                            ignore_index=True)
        df_error.to_excel(os.path.join(path_root, f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx'''), index=False)
        print('- Error Occurred -')
        print()

# = = =  = = =  = = =

list_file = [file for file in sorted(list(os.walk(directory_out))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_out, file))]
for file in list_file:
    os.rename(os.path.join(directory_out, file), os.path.join(directory_out, re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)))

# = = =  = = =  = = =

print('Done ~')

[True] - [请求1次] - [剩余29条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 49. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320962040

[True] - [请求1次] - [剩余28条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 4. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320946090

[True] - [请求1次] - [剩余27条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 28. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-23209f201006



It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.


[True] - [请求1次] - [剩余26条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 13. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232091801003

[True] - [请求1次] - [剩余25条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 16. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092401004

[True] - [请求1次] - [剩余24条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 54. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-23209waa01



It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.


[True] - [请求1次] - [剩余23条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:47] - 46. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092401007

[True] - [请求1次] - [剩余22条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:48] - 11. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232090f010

[True] - [请求1次] - [剩余21条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:48] - 21. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320939196



...

[True] - [请求1次] - [剩余1条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:52] - 50. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320974100



It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

[True] - [请求2次] - [剩余0条] - [crawler_1 (100.00%)] - [2024/11/29 06:17:58] - 20. https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-2320939165

Data Outputting...


It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

Output Finished !

Done ~
